## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

In [2]:
# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saskylakh,RU,2021-10-25 23:48:00,71.9167,114.0833,-0.80,97,100,2.66,overcast clouds
1,1,Graaff-Reinet,ZA,2021-10-25 23:45:04,-32.2522,24.5308,52.92,80,9,2.51,clear sky
2,2,Mandan,US,2021-10-25 23:47:39,46.8267,-100.8896,48.18,72,1,24.16,clear sky
3,3,Beian,NO,2021-10-25 23:48:01,63.6559,9.5683,50.00,93,40,9.22,light rain
4,4,Ribeira Grande,PT,2021-10-25 23:44:54,38.5167,-28.7000,68.74,76,44,20.42,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Rikitea,PF,2021-10-25 23:43:59,-23.1203,-134.9692,75.45,72,69,18.92,broken clouds
12,12,Atuona,PF,2021-10-25 23:48:07,-9.8000,-139.0333,78.12,73,27,16.37,scattered clouds
16,16,Bambous Virieux,MU,2021-10-25 23:48:09,-20.3428,57.7575,72.21,83,40,12.66,scattered clouds
17,17,Esmeraldas,BR,2021-10-25 23:48:10,-19.7625,-44.3139,72.64,91,96,11.10,light rain
20,20,Kawalu,ID,2021-10-25 23:48:12,-7.3817,108.2082,74.93,83,91,1.66,overcast clouds
39,39,Mahebourg,MU,2021-10-25 23:44:12,-20.4081,57.7000,72.25,83,40,12.66,scattered clouds
48,48,Wanning,CN,2021-10-25 23:48:28,18.8003,110.3967,72.57,85,100,9.53,overcast clouds
50,50,Acapulco,MX,2021-10-25 23:48:28,16.8634,-99.8901,78.06,88,100,4.03,overcast clouds
57,57,Pointe-Noire,CG,2021-10-25 23:46:46,-4.7761,11.8635,78.87,89,90,6.91,overcast clouds
59,59,Sao Miguel Do Araguaia,BR,2021-10-25 23:48:34,-13.2750,-50.1628,75.33,84,96,6.24,light rain


In [6]:
len(preferred_cities_df)

182

In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# done in part 3

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,75.45,broken clouds,-23.1203,-134.9692,
12,Atuona,PF,78.12,scattered clouds,-9.8000,-139.0333,
16,Bambous Virieux,MU,72.21,scattered clouds,-20.3428,57.7575,
17,Esmeraldas,BR,72.64,light rain,-19.7625,-44.3139,
20,Kawalu,ID,74.93,overcast clouds,-7.3817,108.2082,
39,Mahebourg,MU,72.25,scattered clouds,-20.4081,57.7000,
48,Wanning,CN,72.57,overcast clouds,18.8003,110.3967,
50,Acapulco,MX,78.06,overcast clouds,16.8634,-99.8901,
57,Pointe-Noire,CG,78.87,overcast clouds,-4.7761,11.8635,
59,Sao Miguel Do Araguaia,BR,75.33,light rain,-13.2750,-50.1628,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel found!")
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel not found... skipping
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel not found... skipping
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel not found... skipping
Hotel found!
Hotel not found... skipping
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel not found... skipping
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Hotel found!
Ho

In [12]:
# clean_hotel_df.dropna(clean_hotel_df['Hotel Name'], inplace=True)

In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df.count()

City                   182
Country                182
Max Temp               182
Current Description    182
Lat                    182
Lng                    182
Hotel Name             182
dtype: int64

In [16]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,75.45,broken clouds,-23.1203,-134.9692,People ThankYou
12,Atuona,PF,78.12,scattered clouds,-9.8000,-139.0333,Villa Enata
16,Bambous Virieux,MU,72.21,scattered clouds,-20.3428,57.7575,Casa Tia Villa
17,Esmeraldas,BR,72.64,light rain,-19.7625,-44.3139,Bom Será - Marcinho e Fernanda
20,Kawalu,ID,74.93,overcast clouds,-7.3817,108.2082,KOSAN UNGU UNSIK
39,Mahebourg,MU,72.25,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
48,Wanning,CN,72.57,overcast clouds,18.8003,110.3967,Paradise Hotel
50,Acapulco,MX,78.06,overcast clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
57,Pointe-Noire,CG,78.87,overcast clouds,-4.7761,11.8635,Kactus Lodge
59,Sao Miguel Do Araguaia,BR,75.33,light rain,-13.2750,-50.1628,Hotel Executivo Palace


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../vacation_search/weatherpy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.35)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))